## Libraries

**Main libraries**

In [1]:
import os
import gc
import pandas as pd 
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from decimal import ROUND_HALF_UP, Decimal
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
import re
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.max_colwidth', 3000)

**Data Pipeline libraries**

In [2]:
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

**MLs Model + Crossvalidation libraries**

In [3]:
from lightgbm import LGBMRegressor
import xgboost
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error,mean_absolute_error

**Optimization libraries**

In [4]:
import optuna

### Local data path

**Local Datasets**

In [ ]:
# %%time
# financials_df = pd.read_csv("train_files/financials.csv")
# options_df = pd.read_csv("train_files/options.csv")
# secondary_stock_prices_df = pd.read_csv("train_files/secondary_stock_prices.csv")
# stock_prices_df = pd.read_csv("train_files/stock_prices.csv")
# trades_df = pd.read_csv("train_files/trades.csv")
# stocks_df = pd.read_csv("stock_list.csv")

**Local Suplemental files**

In [ ]:
# %%time
# financials_info = pd.read_csv("/data_specifications/stock_fin_spec.csv")
# options_info = pd.read_csv("/data_specifications/options_spec.csv")
# stock_prices_info = pd.read_csv("/data_specifications/stock_price_spec.csv")
# trades_info = pd.read_csv("/data_specifications/trades_spec.csv")
# stocks_info = pd.read_csv("/data_specifications/stock_list_spec.csv")

### Kaggle data path

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Datasets**

In [5]:
%%time 
financials_df = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv")
options_df = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/options.csv")
secondary_stock_prices_df = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/secondary_stock_prices.csv")
stock_prices_df = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")
trades_df = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/trades.csv")
stocks_df = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/stock_list.csv")

CPU times: user 30.3 s, sys: 2.83 s, total: 33.1 s
Wall time: 43.1 s


**Suplemental files**

In [ ]:
financials_info = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/stock_fin_spec.csv")
options_info = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/options_spec.csv")
stock_prices_info = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/stock_price_spec.csv")
trades_info = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/trades_spec.csv")
stocks_info = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/stock_list_spec.csv")

## Utils

**Merging stock prices with stock metadata**

In [6]:
def merge_metadata(stock_prices_df, df_stocks):
    """ This function merges the stock price dataset with stock metadata.
        Args:
        stock_prices_df (pd.DataFrame): prices dataframe
        df_stocks (pd.DataFrame): metadata dataframe

        Returns:
        df_prices (pd.DataFrame): merged dataframe
    """
    col = ["SecuritiesCode","Name","Section/Products","NewMarketSegment","33SectorCode","33SectorName","17SectorCode","17SectorName","NewIndexSeriesSizeCode","NewIndexSeriesSize","IssuedShares","MarketCapitalization"]
    df_prices = pd.merge(stock_prices_df, df_stocks[col], on='SecuritiesCode')
    display(df_prices.head())
    return df_prices

**OHLCV Chart plot**

In [ ]:
def plot_candle_with_target(df_prices, stock_code):
    """Plot OHLCV plot with target series.
    
      Args:
        df_prices (pd.DataFrame): prices dataframe
        stock_code: int, code of the stock
    """
    df_ = df_prices.copy() 
    df_ = df_[df_['SecuritiesCode'] == stock_code]
    dates = df_['Date'].values
    ohlc = {
        'open': df_['Open'].values, 
        'high': df_['High'].values, 
        'low': df_['Low'].values, 
        'close': df_['Close'].values
    }
    vol = df_['Volume'].values
    target = df_['Target'].values
    
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, x_title='Date')
    fig.add_trace(go.Candlestick(x=dates, name='OHLC', **ohlc),
                  row=1, col=1)
    fig.add_trace(go.Bar(x=dates, y=vol, name='Volume'),
                  row=2, col=1)
    fig.add_trace(go.Scatter(x=dates, y=target, name='Target'),
                  row=3, col=1)
    fig.update_layout(
        title=f"OHLCV Chart with Target Series (Stock {stock_code})",
    )
    fig.update(layout_xaxis_rangeslider_visible=False)
    fig.show()

In [ ]:
def plot_candle_with_target_adjusted(df_prices, stock_code):
    """Plot OHLCV plot with target series.
    
      Args:
        df_prices (pd.DataFrame): prices dataframe
        stock_code: int, code of the stock
    """
    df_ = df_prices.copy()
    df_ = df_[df_['SecuritiesCode'] == stock_code]
    dates = df_['Date'].values
    ohlc = {
        'open': df_['AdjustedOpen'].values, 
        'high': df_['AdjustedHigh'].values, 
        'low': df_['AdjustedLow'].values, 
        'close': df_['AdjustedClose'].values
    }
    vol = df_['Volume'].values
    target = df_['Target'].values
    
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, x_title='Date')
    fig.add_trace(go.Candlestick(x=dates, name='OHLC', **ohlc),
                  row=1, col=1)
    fig.add_trace(go.Bar(x=dates, y=vol, name='Volume'),
                  row=2, col=1)
    fig.add_trace(go.Scatter(x=dates, y=target, name='Target'),
                  row=3, col=1)
    fig.update_layout(
        title=f"OHLCV Chart with Target Series (Stock {stock_code})",
    )
    fig.update(layout_xaxis_rangeslider_visible=False)
    fig.show()

**Evaluation function (calculates Sharpe Ratio)**

In [7]:
def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

More info about Sharpe Ratio:

https://www.investopedia.com/terms/s/sharperatio.asp

**Adjust Split-Reverse/split price**

In [8]:
def adjust_price(price):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated AdjustedClose
    """
    # transform Date column into datetime
    price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")

    def generate_adjusted_close(df):
        """
        Args:
            df (pd.DataFrame)  : stock_price for a single SecuritiesCode
        Returns:
            df (pd.DataFrame): stock_price with AdjustedClose for a single SecuritiesCode
        """
        # sort data to generate CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        # generate CumulativeAdjustmentFactor
        df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()
        
        
        # generate AdjustedClose
        df.loc[:, "AdjustedClose"] = (
            df["CumulativeAdjustmentFactor"] * df["Close"]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        )) 

        # generate AdjustedOpen
        df.loc[:, "AdjustedOpen"] = (
            df["CumulativeAdjustmentFactor"] * df["Open"]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        
        # generate AdjustedHigh
        df.loc[:, "AdjustedHigh"] = (
            df["CumulativeAdjustmentFactor"] * df["High"]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        
        # generate Low
        df.loc[:, "AdjustedLow"] = (
            df["CumulativeAdjustmentFactor"] * df["Low"]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        
        
        # reverse order
        df = df.sort_values("Date")
        # to fill AdjustedClose, replace 0 into np.nan
        df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
        # forward fill AdjustedClose
       
        #df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()
        
        df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].interpolate(method='linear',limit_direction='backward')
        df.loc[:, "AdjustedOpen"] = df.loc[:, "AdjustedOpen"].interpolate(method='linear',limit_direction='backward')
        df.loc[:, "AdjustedHigh"] = df.loc[:, "AdjustedHigh"].interpolate(method='linear',limit_direction='backward')
        df.loc[:, "AdjustedLow"] = df.loc[:, "AdjustedLow"].interpolate(method='linear',limit_direction='backward')

        return df

    # generate AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted_close).reset_index(drop=True)
    price.drop(columns=['Open', 'High','Low','Close','AdjustmentFactor'],inplace =True)
    #price.set_index("Date", inplace=True)
    return price

**Feature extraction**

In [9]:
def generate_feature(price):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated features
    """
    # transform Date column into datetime

    def generate_feature_close(df):
        """
        Args:
            df (pd.DataFrame)  : stock_price for a single SecuritiesCode
        Returns:
            df (pd.DataFrame): Features for a single SecuritiesCode
        """
        # sort data to generate CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        
        # generate PriceChanges
        period = [5,10,21,33]
        for i in period:
            df[f"pct{i}"] = df["AdjustedClose"].pct_change(i)
            df.loc[:, f"pct{i}"] = df.loc[:, f"pct{i}"].interpolate(method='linear',limit_direction='backward')
            df.loc[:,f"Volatility_{i}"] = np.log(df["AdjustedClose"]).diff().rolling(i).std()
            df.loc[:, f"Volatility_{i}"] = df.loc[:, f"Volatility_{i}"].interpolate(method='linear',limit_direction='backward')

        period_avg = [10,20,50,60]
        for i in period_avg:
         
            # generate SMA
            df[f"SMA_{i}"] = df['AdjustedClose'].rolling(window=i).mean()
            df.loc[:, f"SMA_{i}"] = df.loc[:, f"SMA_{i}"].interpolate(method='linear',limit_direction='backward')
          
            # generate EMA
            df[f"EMA_{i}"] = df['AdjustedClose'].ewm(span=i,adjust=False).mean()
            df.loc[:, f"EMA_{i}"] = df.loc[:, f"EMA_{i}"].interpolate(method='linear',limit_direction='backward')
        
        # reverse order
        df = df.sort_values("Date")

        return df

    # generate AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_feature_close).reset_index(drop=True)
    #price.set_index("Date", inplace=True)
    return price

**Encoding**

In [10]:
def encoding(df):
    """
    Args:
        df (pd.DataFrame): dataframe w/o encoding
    Returns:
        df DataFrame (pd.DataFrame): one hot encoded dataframe
    """
        
    cat_encoder = OneHotEncoder(handle_unknown='ignore')
    train_set_cat_coded = cat_encoder.fit_transform(df[["33SectorName","17SectorName"]])
    ls=[]
    for i in cat_encoder.categories_:
        for j in i:
            ls.append(j)

    train_set_cat_coded_ready = pd.DataFrame(train_set_cat_coded.todense(),columns = ls,index=df.index)
    for i in train_set_cat_coded_ready.columns.to_list():
        train_set_cat_coded_ready[i] = train_set_cat_coded_ready[i].apply(np.int64)
    
    df = pd.concat([df, train_set_cat_coded_ready], axis=1)
    del train_set_cat_coded
    del train_set_cat_coded_ready
    return df

## 1. Exploratory Data Analysis (EDA)

### Stock prices


File: stock_prices.csv

**File Description**

The core file of interest, including the daily closing price for each stock and the target column. Following is column information recorded in stock_price_spec.csv:

In [ ]:
stock_prices_info

In [ ]:
display(stock_prices_df.head(5))

In [ ]:
stock_prices_df.info()

In [ ]:
stock_prices_df.describe()

**Missing values**

In [ ]:
display(pd.isna(stock_prices_df).sum()/len(stock_prices_df)*100)

In [ ]:
missing_high = stock_prices_df[stock_prices_df["High"].isna()]

In [ ]:
display(missing_high.head(5))

In [ ]:
display(missing_high.shape[0])

In [ ]:
missing_high["Date"].nunique()

In [ ]:
missing_high["Date"].value_counts()

In [ ]:
plot_missing_high_df = missing_high["Date"].value_counts().to_frame().reset_index()
plot_missing_high_df.rename(columns = {'index':'Date', 'Date':'Count'}, inplace = True)
plot_missing_high_df.head()

2020-10-01 is the day with the most amount of missing data

In [ ]:
plot_missing_high_df["Date"] = pd.to_datetime(plot_missing_high_df["Date"])
plot_missing_high_df.sort_values(by="Date",inplace=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=plot_missing_high_df["Date"], 
                         y=plot_missing_high_df["Count"], 
                         mode='lines'))

fig.update_layout(
    title=f"Stocks without Prices Count per Date",
    xaxis_title="Date",
    yaxis_title="Count",
)

In [ ]:
missing_Target = stock_prices_df[stock_prices_df["Target"].isna()]

In [ ]:
display(missing_Target.head(5))

In [ ]:
display(missing_Target.shape[0])

In [ ]:
missing_Target["Date"].nunique()

In [ ]:
missing_Target["Date"].value_counts()

In [ ]:
plot_missing_target_df = missing_Target["Date"].value_counts().to_frame().reset_index()
plot_missing_target_df.rename(columns = {'index':'Date', 'Date':'Count'}, inplace = True)
plot_missing_target_df

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=plot_missing_target_df["Date"], 
                         y=plot_missing_target_df["Count"], 
                         mode='lines'))

fig.update_layout(
    title=f"Stocks without Target Count per Date",
    xaxis_title="Date",
    yaxis_title="Count",
)

### Stocks

stock_list.csv


**File Description**

Stock metadata. Following is column information recorded in stock_list_spec.csv:

In [ ]:
display(stocks_info)

In [ ]:
display(stocks_df.head(5))

In [ ]:
stocks_df.info()

In [ ]:
display(pd.isna(stocks_df).sum()/len(stocks_df)*100)

In [ ]:
stocks_missing_Date = stocks_df[stocks_df["TradeDate"].isna()]
stocks_missing_Date.head()

In [ ]:
stocks_missing_segment = stocks_df[stocks_df["NewMarketSegment"].isna()]
stocks_missing_segment.head()

In [ ]:
stocks_df["Universe0"].value_counts()

In [ ]:
stocks_df["33SectorName"].value_counts()

In [ ]:
sector = stocks_df["33SectorName"].value_counts().to_frame()
sector.reset_index(inplace=True)
sector["percentage"] = sector["33SectorName"]/sector["33SectorName"].sum()*100
sector.head(5)

In [ ]:
plt.rcParams["figure.figsize"] = (30,10)
plt.pie(sector["percentage"], labels = sector["index"])
plt.show()

In [ ]:
stocks_df["Section/Products"].value_counts()

In [ ]:
section = stocks_df["Section/Products"].value_counts().to_frame()
section.reset_index(inplace=True)
section["percentage"] = section["Section/Products"]/section["Section/Products"].sum()*100
section

In [ ]:
section["percentage"] = section["Section/Products"]/section["Section/Products"].sum()*100

In [ ]:
plt.rcParams["figure.figsize"] = (30,10)
plt.pie(section["percentage"], labels = section["index"])
plt.show()

In [ ]:
del sector
del section
gc.collect()

## Selecting the top 2000 Stocks (Universe0 Flag = True)

In [11]:
stocks2000_df = stocks_df[stocks_df["Universe0"]]
display(stocks2000_df.head())

,SecuritiesCode,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,TradeDate,Close,IssuedShares,MarketCapitalization,Universe0
0,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
19,1332,20211230,"Nippon Suisan Kaisha,Ltd.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,4,TOPIX Mid400,20211230.0,543.0,312430277.0,1.696496e+11,True
20,1333,20211230,Maruha Nichiro Corporation,First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,4,TOPIX Mid400,20211230.0,2405.0,52656910.0,1.266399e+11,True
37,1375,20211230,"YUKIGUNI MAITAKE CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,6,TOPIX Small 1,20211230.0,1196.0,39910700.0,4.773320e+10,True
38,1376,20211230,"KANEKO SEEDS CO.,LTD.",First Section (Domestic),Standard Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,1504.0,11772626.0,1.770603e+10,True


In [ ]:
stocks2000_df.info()

In [ ]:
stocks2000_df["33SectorName"].value_counts()

In [ ]:
sector2000 = stocks2000_df["33SectorName"].value_counts().to_frame()
sector2000.reset_index(inplace=True)
sector2000["percentage"] = sector2000["33SectorName"]/sector2000["33SectorName"].sum()*100
sector2000.head(5)

In [ ]:
plt.rcParams["figure.figsize"] = (40,10)
plt.pie(sector2000["percentage"], labels = sector2000["index"])
plt.show()

In [ ]:
section2000 = stocks2000_df["Section/Products"].value_counts().to_frame()
section2000.reset_index(inplace=True)
section2000["percentage"] = section2000["Section/Products"]/section2000["Section/Products"].sum()*100
section2000

In [ ]:
plt.rcParams["figure.figsize"] = (30,10)
plt.pie(section2000["percentage"], labels = section2000["index"])
plt.show()

In [ ]:
del stocks_df
del section2000
del sector2000
gc.collect()

## Merging Stock metadate with stock price

In [12]:
df_prices =  merge_metadata(stock_prices_df, stocks2000_df)

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,IssuedShares,MarketCapitalization
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,10928283.0,3.365911e+10
1,20170105_1301,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,False,0.002920,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,10928283.0,3.365911e+10
2,20170106_1301,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,NaN,False,-0.001092,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,10928283.0,3.365911e+10
3,20170110_1301,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,NaN,False,-0.005100,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,10928283.0,3.365911e+10
4,20170111_1301,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,NaN,False,-0.003295,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,10928283.0,3.365911e+10


**Calculating Target variable statistics**

In [ ]:
mean_securities_df = df_prices.groupby(["SecuritiesCode"])["Target"].mean()
total_mean_securities = mean_securities_df.mean()
total_mean_securities

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.histplot(data=mean_securities_df.values, bins=100,
             ax=ax)
ax.axvline(x=total_mean_securities, color='red', linestyle='dotted', linewidth=2, 
           label='Mean')
ax.set_title("Target Mean Distibution Securities\n"
             f"Min {round(mean_securities_df.min(), 4)} | "
             f"Max {round(mean_securities_df.max(), 4)} | "
             f"Skewness {round(mean_securities_df.skew(), 2)} | "
             f"Kurtosis {round(mean_securities_df.kurtosis(), 2)}")
ax.set_xlabel("Target Mean")
ax.set_ylabel("Date Count")
ax.legend()
plt.show()

In [ ]:
mean_date_df = df_prices.groupby(["Date"])["Target"].mean()
total_mean_date = mean_date_df.mean()
total_mean_date

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.histplot(data=mean_date_df.values, bins=100,
             ax=ax)
ax.axvline(x=total_mean_date, color='red', linestyle='dotted', linewidth=2, 
           label='Mean')
ax.set_title("Target Mean Distibution Date\n"
             f"Min {round(mean_date_df.min(), 4)} | "
             f"Max {round(mean_date_df.max(), 4)} | "
             f"Skewness {round(mean_date_df.skew(), 2)} | "
             f"Kurtosis {round(mean_date_df.kurtosis(), 2)}")
ax.set_xlabel("Target Mean")
ax.set_ylabel("Date Count")
ax.legend()
plt.show()

## Grupying the Target by Sector and Section

### Target vs Sector33

In [ ]:
target_sector = df_prices.groupby(["33SectorName"])["Target"].mean()
target_sector.sort_values(inplace=True, ascending=False)


In [ ]:
target_sector = target_sector.to_frame()
target_sector.reset_index(inplace= True)

In [ ]:
fig, ax = plt.subplots(figsize = (12,6))    
fig = sns.barplot(x = "33SectorName", y = "Target", data = target_sector, ax=ax)
ax.set_title("Target Mean of Sectors")
ax.tick_params(axis='x', rotation=90)

### Target vs Section

In [ ]:
target_section = df_prices.groupby(["Section/Products"])["Target"].mean()
target_section.sort_values(inplace=True, ascending=False)


In [ ]:
target_section = target_section.to_frame()
target_section.reset_index(inplace= True)

In [ ]:
fig, ax = plt.subplots(figsize = (12,6))    
fig = sns.barplot(x = "Section/Products", y = "Target", data = target_section, ax=ax)
ax.set_title("Target Mean of Sections")
ax.tick_params(axis='x', rotation=90)

### Secondary stock prices

secondary_stock_prices.csv

**File Description**

Securities with low liquidity (few opportunities to trade). Following is column information recorded in stock_price_spec.csv:

In [ ]:
stock_prices_info

In [ ]:
display(secondary_stock_prices_df.head(5))

In [ ]:
secondary_stock_prices_df.info()

In [ ]:
display(pd.isna(secondary_stock_prices_df).sum()/len(secondary_stock_prices_df)*100)

**Missing values**

In [ ]:
missing_secondary_high = secondary_stock_prices_df[secondary_stock_prices_df["High"].isna()]
display(missing_secondary_high.head(5))

In [ ]:
missing_secondary_high["Date"].nunique()

In [ ]:
plot_missing_secondary_high_df = missing_secondary_high["Date"].value_counts().to_frame().reset_index()
plot_missing_secondary_high_df.rename(columns = {'index':'Date', 'Date':'Count'}, inplace = True)
plot_missing_secondary_high_df.head()

In [ ]:
del secondary_stock_prices_df
gc.collect()

As happened with the primary stock list 2020-10-01 is the day with the most amount of missing data


**NOTE:** Secondary stock prices price movement is correlated with the main stock price movement. Due to the limited amount dedicated to this project, this data will not be included in the prediction model

## Trades

trades.csv

**File Description**

Aggregated summary of trading volumes from the previous business week. Following is column information recorded in trades_spec.csv:

In [ ]:
trades_info

In [ ]:
display(trades_df.head(5))

**NOTE:** Trade volumen info can be used to predict the Target value. Due to the limited amount dedicated to this project, this data will not be included in the prediction model

### Financials 

financials.csv

**File Description**

Financial information about stock products. Following is column information recorded in stock_fin_spec.csv:

In [ ]:
financials_info

In [ ]:
financials_df.head()

In [ ]:
financials_df.info()

In [ ]:
display(pd.isna(financials_df).sum()/len(financials_df)*100)

Notably a high amount of data is missing in financials_df

**NOTE:** Leveraging the financial data in the Fundamental analysis is one popular method in target calculation. Due to the limited amount dedicated to this project, this data will not be included in the prediction model

### Options

options.csv

**File Description**

Data on the status of a variety of options based on the broader market. Many options include implicit predictions of the future price of the stock market and so may be of interest even though the options are not scored directly. Following is column information recorded in options_spec.csv:


In [ ]:
options_info

In [ ]:
options_df.head()

In [ ]:
options_df.info()

**NOTE:** Info provided by Options (for instance the put/call volumen ratio) is interesting to determine the market feeling about a particular derivated product. Due to the limited amount dedicated to this project, this data will not be included in the prediction model

## 2. Data wrangling
- Adjust OHLC prices (Split- Reverse/split)
- Input missing values


#### Generate adjusted OHLC prices
Generate adjusted OHLC prices using AdjustmentFactor attribute. This should reduce historical price gap caused by split/reverse-split.

Furthermore, adjust_price() also inputs the missing data

In [13]:
df_prices_adj = adjust_price(df_prices)
df_prices_adj.head(5)

,RowId,Date,SecuritiesCode,Volume,ExpectedDividend,SupervisionFlag,Target,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,IssuedShares,MarketCapitalization,CumulativeAdjustmentFactor,AdjustedClose,AdjustedOpen,AdjustedHigh,AdjustedLow
0,20170104_1301,2017-01-04,1301,31400,NaN,False,0.000730,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,10928283.0,3.365911e+10,1.0,2742.0,2734.0,2755.0,2730.0
1,20170105_1301,2017-01-05,1301,17900,NaN,False,0.002920,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,10928283.0,3.365911e+10,1.0,2738.0,2743.0,2747.0,2735.0
2,20170106_1301,2017-01-06,1301,19900,NaN,False,-0.001092,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,10928283.0,3.365911e+10,1.0,2740.0,2734.0,2744.0,2720.0
3,20170110_1301,2017-01-10,1301,24200,NaN,False,-0.005100,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,10928283.0,3.365911e+10,1.0,2748.0,2745.0,2754.0,2735.0
4,20170111_1301,2017-01-11,1301,9300,NaN,False,-0.003295,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,10928283.0,3.365911e+10,1.0,2745.0,2748.0,2752.0,2737.0


In [ ]:
df_prices_adj["CumulativeAdjustmentFactor"].value_counts()

In [ ]:
df_prices_adj.info()

**Less missing values**

In [ ]:
display(pd.isna(df_prices_adj).sum()/len(df_prices_adj)*100)

**Now the OHTC chart shows continuity**

Before adjustment

In [ ]:
plot_candle_with_target(df_prices, 9726) 

After adjustment

In [ ]:
plot_candle_with_target_adjusted(adjust_price(df_prices), 9726) 

Before adjustment

In [ ]:
plot_candle_with_target(df_prices, 4582)

After adjustment

In [ ]:
plot_candle_with_target_adjusted(adjust_price(df_prices), 4582) 

Before adjustment

In [ ]:
plot_candle_with_target(df_prices, 1805)

After adjustment

In [ ]:
plot_candle_with_target_adjusted(adjust_price(df_prices), 1805) 

## 3. Feature Engineering


**Correlogram to see if there is any autocorrelation**

In [ ]:
df_9726 =df_prices_adj[df_prices_adj["SecuritiesCode"]==9726]

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]
pd.plotting.autocorrelation_plot(df_9726["AdjustedClose"])

In [ ]:
pd.plotting.autocorrelation_plot(df_9726["Target"])

It certainly look like we are dealing with a random walk, as there are no indications of any autocorrelation for any lag.

Basically our LSTM found nothing of any real value to model and thus took the average value, along with a slight slope; we would have been just as well off with an extremely simplistic model of the form

C
l
o
s
e
t
∝
C
l
o
s
e
(
t
−
1
)

Close Price shows autocorrelation but stock return not. Therefore, the target variable itself will not give us much info about future stock movements... At this point the original idea of using a LSTM model was discarded at it was proceed to continue with the feature engineering.

### Feature Engineering

**From adjusted prices (individual)**
- Price changes
- Moving average
- Exponential moving average
- Volatility
- One hot encoding
- Standarization?

**Price changes**


In [ ]:
period = [5,10,21,33]

In [ ]:
for i in period:
    df_9726[f"pct{i}"] = df_9726["AdjustedClose"].pct_change(i)
display(df_9726.head(5))

**Simple Moving Average (SMA)**

In [ ]:
period_avg = [10,20,50,60]

In [ ]:
for i in period_avg:
    df_9726[f"SMA_{i}"] = df_9726['AdjustedClose'].rolling(window=i).mean()

display(df_9726.tail(5))

**Exponential Moving Average (EMA)**

In [ ]:
for i in period_avg:
    df_9726[f"EMA_{i}"] = df_9726['AdjustedClose'].ewm(span=i,adjust=False).mean()

display(df_9726.tail(5))

Plotting the new features

In [ ]:
col_avg = ["AdjustedClose","SMA_10","SMA_20","SMA_50","SMA_60","EMA_10","EMA_20","EMA_20","EMA_60"]

In [ ]:
df_9726Avg = df_9726[col_avg]
df_9726Avg.head()

In [ ]:
plt.rcParams["figure.figsize"] = (30,10)
df_9726Avg.plot(title = "Avg analysis for Security Code")

In [ ]:
col_SMA = ["AdjustedClose","SMA_10","SMA_20","SMA_50","SMA_60"]

In [ ]:
df_9726SMA = df_9726[col_SMA]
df_9726SMA.head()

In [ ]:
df_9726SMA.plot(title = "Avg analysis for Security Code")

In [ ]:
col_EMA = ["AdjustedClose","EMA_10","EMA_20","EMA_50","EMA_60"]

In [ ]:
df_9726EMA = df_9726[col_EMA]
df_9726EMA.head()

In [ ]:
df_9726EMA.plot(title = "Avg analysis for Security Code")

In [14]:
df_prices_feat = generate_feature(df_prices_adj)
df_prices_feat.drop(["RowId","ExpectedDividend","Name","Section/Products","SupervisionFlag","NewMarketSegment","33SectorCode","17SectorCode","NewIndexSeriesSizeCode","NewIndexSeriesSize","IssuedShares","MarketCapitalization"],axis=1,inplace=True)
df_prices_feat.head(5)

,Date,SecuritiesCode,Volume,Target,33SectorName,17SectorName,CumulativeAdjustmentFactor,AdjustedClose,AdjustedOpen,AdjustedHigh,AdjustedLow,pct5,Volatility_5,pct10,Volatility_10,pct21,Volatility_21,pct33,Volatility_33,SMA_10,EMA_10,SMA_20,EMA_20,SMA_50,EMA_50,SMA_60,EMA_60
0,2017-01-04,1301,31400,0.000730,"Fishery, Agriculture and Forestry",FOODS,1.0,2742.0,2734.0,2755.0,2730.0,0.004028,0.002969,0.015932,0.003732,0.008088,0.004796,-0.048577,0.005294,2725.0,2733.158949,2725.75,2741.853831,2847.54,2815.764772,2891.566667,2838.160046
1,2017-01-05,1301,17900,0.002920,"Fishery, Agriculture and Forestry",FOODS,1.0,2738.0,2743.0,2747.0,2735.0,0.005878,0.003178,0.000365,0.006179,0.004771,0.004813,-0.064571,0.005795,2720.7,2731.194271,2725.10,2741.838444,2855.90,2818.775579,2894.800000,2841.419708
2,2017-01-06,1301,19900,-0.001092,"Fishery, Agriculture and Forestry",FOODS,1.0,2740.0,2734.0,2744.0,2720.0,0.013314,0.003734,0.007353,0.006468,0.003295,0.004838,-0.064208,0.005799,2720.6,2729.681887,2724.20,2742.242491,2865.04,2822.072541,2899.333333,2844.925461
3,2017-01-10,1301,24200,-0.005100,"Fishery, Agriculture and Forestry",FOODS,1.0,2748.0,2745.0,2754.0,2735.0,0.023083,0.002383,0.007701,0.006448,0.004019,0.004818,-0.066893,0.005834,2718.6,2727.388973,2723.45,2742.478543,2873.94,2825.422441,2903.833333,2848.482256
4,2017-01-11,1301,9300,-0.003295,"Fishery, Agriculture and Forestry",FOODS,1.0,2745.0,2748.0,2752.0,2737.0,0.018931,0.004005,0.005126,0.006484,0.002923,0.004813,-0.068228,0.005815,2716.5,2722.808745,2722.60,2741.897337,2881.98,2828.582540,2908.116667,2851.888435


In [15]:
df_prices_feat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2332531 entries, 0 to 2332530
Data columns (total 27 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   Date                        datetime64[ns]
 1   SecuritiesCode              int64         
 2   Volume                      int64         
 3   Target                      float64       
 4   33SectorName                object        
 5   17SectorName                object        
 6   CumulativeAdjustmentFactor  float64       
 7   AdjustedClose               float64       
 8   AdjustedOpen                float64       
 9   AdjustedHigh                float64       
 10  AdjustedLow                 float64       
 11  pct5                        float64       
 12  Volatility_5                float64       
 13  pct10                       float64       
 14  Volatility_10               float64       
 15  pct21                       float64       
 16  Volatility_21     

In [ ]:
del df_prices_adj
del df_9726
del df_9726EMA
del df_9726SMA
del options_df
del financials_df
del trades_df
del financials_info
del options_info
del stock_prices_info
del trades_info
del stocks_info
gc.collect()

In [ ]:
del target_sector
del target_section
gc.collect()

**One Hot encoding Stock Catagorical data**
- SectorCode33
- Section

In [17]:
cat_encoder = OneHotEncoder(handle_unknown='ignore')
train_set_cat_coded = cat_encoder.fit_transform(df_prices_feat[["33SectorName","17SectorName"]])
train_set_cat_coded.shape

(2332531, 50)

In [18]:
ls=[]
for i in cat_encoder.categories_:
    for j in i:
        ls.append(j)

In [19]:
train_set_cat_coded_ready = pd.DataFrame(train_set_cat_coded.todense(),columns = ls,index=df_prices_feat.index)
train_set_cat_coded_ready

,Air Transportation,Banks,Chemicals,Construction,Electric Appliances,Electric Power and Gas,"Fishery, Agriculture and Forestry",Foods,Glass and Ceramics Products,Information & Communication,Insurance,Iron and Steel,Land Transportation,Machinery,Marine Transportation,Metal Products,Mining,Nonferrous Metals,Oil and Coal Products,Other Financing Business,Other Products,Pharmaceutical,Precision Instruments,Pulp and Paper,Real Estate,Retail Trade,Rubber Products,Securities and Commodities Futures,Services,Textiles and Apparels,Transportation Equipment,Warehousing and Harbor Transportation Service,Wholesale Trade,AUTOMOBILES & TRANSPORTATION EQUIPMENT,BANKS,COMMERCIAL & WHOLESALE TRADE,CONSTRUCTION & MATERIALS,ELECTRIC APPLIANCES & PRECISION INSTRUMENTS,ELECTRIC POWER & GAS,ENERGY RESOURCES,FINANCIALS （EX BANKS）,FOODS,"IT & SERVICES, OTHERS",MACHINERY,PHARMACEUTICAL,RAW MATERIALS & CHEMICALS,REAL ESTATE,RETAIL TRADE,STEEL & NONFERROUS METALS,TRANSPORTATION & LOGISTICS
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2332527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2332528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2332529,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [20]:
for i in train_set_cat_coded_ready.columns.to_list():
    train_set_cat_coded_ready[i] = train_set_cat_coded_ready[i].apply(np.uint8)

In [21]:
df_prices_feat = pd.concat([df_prices_feat, train_set_cat_coded_ready], axis=1)

In [22]:
del train_set_cat_coded
del train_set_cat_coded_ready
gc.collect()

23

**Selecting features for the model**

In [23]:
col = ["Date","SecuritiesCode","Target","Volume","AdjustedClose","AdjustedHigh","AdjustedOpen","AdjustedLow","pct5","pct10","pct21","pct33","Volatility_5","Volatility_10","Volatility_21","Volatility_33","SMA_10","SMA_20","SMA_50","SMA_60","EMA_10","EMA_20","EMA_50"] +ls

In [24]:
X = df_prices_feat[col]
X=X.dropna().sort_values(['Date','SecuritiesCode'])

**Correlation Matrix**

In [ ]:
corr_mat = np.abs(X.corr())
fig, ax = plt.subplots(figsize=(20,20))
sns.heatmap(corr_mat,square = True, ax = ax)

**Removing special characters in the column names**

In [25]:
X = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

**Drop labels**

In [26]:
y= X["Target"].to_numpy()
X=X.drop(["Target"],axis=1)

### Standarize Values

A data pipeline was originally created to standarize the input attribute for the model. After several tries, it was concluded that the model works better without standarized values.

In [ ]:
num_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
    ])

In [ ]:
num_attribs = ['Volume','AdjustedClose',"AdjustedHigh","AdjustedOpen","AdjustedLow","pct5","pct10","pct21","pct33","Volatility_5","Volatility_10","Volatility_21","Volatility_33",'SMA_10','SMA_20','SMA_50','SMA_60','EMA_10','EMA_20','EMA_50']

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs)
    ])


## Model Selection

Calculate the average Sharpe ratio for the following ML models.

- LGBMRegressor
- XGBoost Regressor
- Random Forest Regressor

The model giving the **highest** Sharpe ratio will be selected for the final model.

**Create list to store all Sharpe ratio for different models**

In [ ]:
model_performance = []

### Time based Crossvalidation

**LGBMRegressor**

In [ ]:
ts_fold = TimeSeriesSplit(n_splits=10,gap=10000)

In [ ]:
feat_importance=pd.DataFrame()
sharpe_ratio=[]

In [ ]:
%%time
for fold, (train_idx, val_idx) in enumerate(ts_fold.split(X, y)):
    
    print(f"\n========================== Fold {fold+1} ==========================")
        
    X_train, y_train = X.iloc[train_idx,:], y[train_idx]
    X_valid, y_val = X.iloc[val_idx,:], y[val_idx]
    
    print("Train Date range: {} to {}".format(X_train.Date.min(),X_train.Date.max()))
    print("Valid Date range: {} to {}".format(X_valid.Date.min(),X_valid.Date.max()))
    
    
    ##### TRAIN standarization pipeline. See comment above
    """
    pipeline_X_df = full_pipeline.fit_transform(X_train)
    pipeline_X_df = pd.DataFrame(pipeline_X_df, columns = num_attribs)
    
    pipeline_X_df.reset_index(drop = True, inplace=True)
    X_train.reset_index(drop = True, inplace=True)

    X_train.drop(num_attribs,axis=1,inplace=True)
    X_train = pd.concat([X_train,pipeline_X_df], axis=1)
    #"""
    X_train= X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    
    ##### VALIDATION. See comment above
    """
    pipeline_val_df = full_pipeline.transform(X_valid)
    pipeline_val_df = pd.DataFrame(pipeline_val_df, columns = num_attribs)
    pipeline_val_df.reset_index(drop = True, inplace=True)
    X_valid.reset_index(drop = True, inplace=True)
    X_valid.drop(num_attribs,axis=1,inplace=True)
    X_valid = pd.concat([X_valid,pipeline_val_df], axis=1)
    
   # """
    X_valid= X_valid.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    
    X_train.drop(['Date','SecuritiesCode'], axis=1, inplace=True)
    X_val=X_valid[X_valid.columns[~X_valid.columns.isin(['Date','SecuritiesCode'])]]
    val_dates=X_valid.Date.unique()[1:-1]
    print("\nTrain Shape: {} {}, Valid Shape: {} {}".format(X_train.shape, y_train.shape, X_val.shape, y_val.shape))
    
    #gbm = LGBMRegressor(**params)
    gbm = LGBMRegressor().fit(X_train, y_train, 
                                      eval_set=[(X_train, y_train), (X_val, y_val)],
                                      verbose=300, 
                                      eval_metric=['mae','mse'])
    y_pred = gbm.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mae = mean_absolute_error(y_val, y_pred)
    feat_importance["Importance_Fold"+str(fold)]=gbm.feature_importances_
    feat_importance.set_index(X_train.columns, inplace=True)
        
    rank=[]
    X_val_df=X_valid[X_valid.Date.isin(val_dates)]
    for i in X_val_df.Date.unique():
        temp_df = X_val_df[X_val_df.Date == i].drop(['Date','SecuritiesCode'],axis=1)
        temp_df["pred"] = gbm.predict(temp_df)
        temp_df["Rank"] = (temp_df["pred"].rank(method="first", ascending=False)-1).astype(int)
        rank.append(temp_df["Rank"].values)

    stock_rank=pd.Series([x for y in rank for x in y], name="Rank")
    df=pd.concat([X_val_df.reset_index(drop=True),stock_rank,
                  df_prices_feat[df_prices_feat.Date.isin(val_dates)]['Target'].reset_index(drop=True)], axis=1)
    sharpe=calc_spread_return_sharpe(df)
    sharpe_ratio.append(sharpe)
    print("Valid Sharpe: {}, RMSE: {}, MAE: {}".format(sharpe,rmse,mae))
    
    del X_train, y_train,  X_val, y_val ##, pipeline_val_df, pipeline_X_df
    gc.collect()
    
print("\nAverage cross-validation Sharpe Ratio: {:.4f}, standard deviation = {:.2f}.".format(np.mean(sharpe_ratio),np.std(sharpe_ratio)))

In [ ]:
feat_importance['avg'] = feat_importance.mean(axis=1)
feat_importance = feat_importance.sort_values(by='avg',ascending=True)
feat_importance = feat_importance.tail(20)
pal=sns.color_palette("plasma_r", 29).as_hex()[2:]

fig=go.Figure()
for i in range(len(feat_importance.index)):
    fig.add_shape(dict(type="line", y0=i, y1=i, x0=0, x1=feat_importance['avg'][i], 
                       line_color=pal[::-1][i],opacity=0.7,line_width=4))
    fig.add_trace(go.Scatter(x=feat_importance['avg'], y=feat_importance.index, mode='markers', 
                             marker_color=pal[::-1], marker_size=8,
                             hovertemplate='%{y} Importance = %{x:.0f}<extra></extra>'))
    fig.update_layout(title='Overall Feature Importance', 
                      xaxis=dict(title='Average Importance',zeroline=False),
                      yaxis_showgrid=False, margin=dict(l=120,t=80),
                      height=700, width=800)
fig.show()

In [ ]:
model_performance.append({"model":"LGBMRegressor","Mean Sharpe Ratio":np.mean(sharpe_ratio),"SD Sharpe Ratio":np.std(sharpe_ratio)})

**XGBoost Regressor**

In [ ]:
ts_fold = TimeSeriesSplit(n_splits=10,gap=10000)
feat_importance=pd.DataFrame()
sharpe_ratio=[]

In [ ]:
%%time
for fold, (train_idx, val_idx) in enumerate(ts_fold.split(X, y)):
    
    print(f"\n========================== Fold {fold+1} ==========================")
        
    X_train, y_train = X.iloc[train_idx,:], y[train_idx]
    X_valid, y_val = X.iloc[val_idx,:], y[val_idx]
    
    print("Train Date range: {} to {}".format(X_train.Date.min(),X_train.Date.max()))
    print("Valid Date range: {} to {}".format(X_valid.Date.min(),X_valid.Date.max()))
    
    
    ##### TRAIN
    """
    pipeline_X_df = full_pipeline.fit_transform(X_train)
    pipeline_X_df = pd.DataFrame(pipeline_X_df, columns = num_attribs)
    
    pipeline_X_df.reset_index(drop = True, inplace=True)
    X_train.reset_index(drop = True, inplace=True)

    X_train.drop(num_attribs,axis=1,inplace=True)
    X_train = pd.concat([X_train,pipeline_X_df], axis=1)
    #"""
    X_train= X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    
    ##### VALIDATION
    """
    pipeline_val_df = full_pipeline.transform(X_valid)
    pipeline_val_df = pd.DataFrame(pipeline_val_df, columns = num_attribs)
    pipeline_val_df.reset_index(drop = True, inplace=True)
    X_valid.reset_index(drop = True, inplace=True)
    X_valid.drop(num_attribs,axis=1,inplace=True)
    X_valid = pd.concat([X_valid,pipeline_val_df], axis=1)
    
   # """
    X_valid= X_valid.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    
    X_train.drop(['Date','SecuritiesCode'], axis=1, inplace=True)
    X_val=X_valid[X_valid.columns[~X_valid.columns.isin(['Date','SecuritiesCode'])]]
    val_dates=X_valid.Date.unique()[1:-1]
    print("\nTrain Shape: {} {}, Valid Shape: {} {}".format(X_train.shape, y_train.shape, X_val.shape, y_val.shape))
    
    xgb = xgboost.XGBRegressor()
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mae = mean_absolute_error(y_val, y_pred)
    feat_importance["Importance_Fold"+str(fold)]=xgb.feature_importances_
    feat_importance.set_index(X_train.columns, inplace=True)
        
    rank=[]
    X_val_df=X_valid[X_valid.Date.isin(val_dates)]
    for i in X_val_df.Date.unique():
        temp_df = X_val_df[X_val_df.Date == i].drop(['Date','SecuritiesCode'],axis=1)
        temp_df["pred"] = xgb.predict(temp_df)
        temp_df["Rank"] = (temp_df["pred"].rank(method="first", ascending=False)-1).astype(int)
        rank.append(temp_df["Rank"].values)

    stock_rank=pd.Series([x for y in rank for x in y], name="Rank")
    df=pd.concat([X_val_df.reset_index(drop=True),stock_rank,
                  df_prices_feat[df_prices_feat.Date.isin(val_dates)]['Target'].reset_index(drop=True)], axis=1)
    sharpe=calc_spread_return_sharpe(df)
    sharpe_ratio.append(sharpe)
    print("Valid Sharpe: {}, RMSE: {}, MAE: {}".format(sharpe,rmse,mae))
    
    del X_train, y_train,  X_val, y_val ##, pipeline_val_df, pipeline_X_df
    gc.collect()
    
print("\nAverage cross-validation Sharpe Ratio: {:.4f}, standard deviation = {:.2f}.".format(np.mean(sharpe_ratio),np.std(sharpe_ratio)))

In [ ]:
feat_importance['avg'] = feat_importance.mean(axis=1)
feat_importance = feat_importance.sort_values(by='avg',ascending=True)
feat_importance = feat_importance.tail(20)
pal=sns.color_palette("plasma_r", 29).as_hex()[2:]

fig=go.Figure()
for i in range(len(feat_importance.index)):
    fig.add_shape(dict(type="line", y0=i, y1=i, x0=0, x1=feat_importance['avg'][i], 
                       line_color=pal[::-1][i],opacity=0.7,line_width=4))
    fig.add_trace(go.Scatter(x=feat_importance['avg'], y=feat_importance.index, mode='markers', 
                             marker_color=pal[::-1], marker_size=8,
                             hovertemplate='%{y} Importance = %{x:.0f}<extra></extra>'))
    fig.update_layout(title='Overall Feature Importance', 
                      xaxis=dict(title='Average Importance',zeroline=False),
                      yaxis_showgrid=False, margin=dict(l=120,t=80),
                      height=700, width=800)
fig.show()

In [ ]:
model_performance.append({"model":"XGBRegressor","Mean Sharpe Ratio":np.mean(sharpe_ratio),"SD Sharpe Ratio":np.std(sharpe_ratio)})

**Random Forest Regressor**

**Disclaimer**:

Random Forest Regressor was tried several time. The trainning was extremely slow plus the model would crash several times at the 3-Fold. Somehow the model got stuck. Therefore, the code is maintained on the notebook just for educational purposed. The code is commented though.

In [ ]:
# %%time
# ts_fold = TimeSeriesSplit(n_splits=5,gap=5000)
# feat_importance=pd.DataFrame()
# sharpe_ratio=[]

# for fold, (train_idx, val_idx) in enumerate(ts_fold.split(X, y)):
    
#     print(f"\n========================== Fold {fold+1} ==========================")
        
#     X_train, y_train = X.iloc[train_idx,:], y[train_idx]
#     X_valid, y_val = X.iloc[val_idx,:], y[val_idx]
    
#     print("Train Date range: {} to {}".format(X_train.Date.min(),X_train.Date.max()))
#     print("Valid Date range: {} to {}".format(X_valid.Date.min(),X_valid.Date.max()))
    
    
#     ##### TRAIIIIIIIIIIN
#     """
#     pipeline_X_df = full_pipeline.fit_transform(X_train)
#     pipeline_X_df = pd.DataFrame(pipeline_X_df, columns = num_attribs)
    
#     pipeline_X_df.reset_index(drop = True, inplace=True)
#     X_train.reset_index(drop = True, inplace=True)

#     X_train.drop(num_attribs,axis=1,inplace=True)
#     X_train = pd.concat([X_train,pipeline_X_df], axis=1)
#     #"""
    
    
#     ##### VALIDATION
#     """
#     pipeline_val_df = full_pipeline.transform(X_valid)
#     pipeline_val_df = pd.DataFrame(pipeline_val_df, columns = num_attribs)
#     pipeline_val_df.reset_index(drop = True, inplace=True)
#     X_valid.reset_index(drop = True, inplace=True)
#     X_valid.drop(num_attribs,axis=1,inplace=True)
#     X_valid = pd.concat([X_valid,pipeline_val_df], axis=1)
    
#     """
    
#     X_train= X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
#     X_valid= X_valid.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    
#     X_train.drop(['Date','SecuritiesCode'], axis=1, inplace=True)
#     X_val=X_valid[X_valid.columns[~X_valid.columns.isin(['Date','SecuritiesCode'])]]
#     val_dates=X_valid.Date.unique()[1:-1]
#     print("\nTrain Shape: {} {}, Valid Shape: {} {}".format(X_train.shape, y_train.shape, X_val.shape, y_val.shape))
    
#     regr = RandomForestRegressor()
#     regr.fit(X_train, y_train)
#     y_pred = regr.predict(X_val)
#     rmse = np.sqrt(mean_squared_error(y_val, y_pred))
#     mae = mean_absolute_error(y_val, y_pred)
#     feat_importance["Importance_Fold"+str(fold)]=regr.feature_importances_
#     feat_importance.set_index(X_train.columns, inplace=True)
        
#     rank=[]
#     X_val_df=X_valid[X_valid.Date.isin(val_dates)]
#     for i in X_val_df.Date.unique():
#         temp_df = X_val_df[X_val_df.Date == i].drop(['Date','SecuritiesCode'],axis=1)
#         temp_df["pred"] = regr.predict(temp_df)
#         temp_df["Rank"] = (temp_df["pred"].rank(method="first", ascending=False)-1).astype(int)
#         rank.append(temp_df["Rank"].values)

#     stock_rank=pd.Series([x for y in rank for x in y], name="Rank")
#     df=pd.concat([X_val_df.reset_index(drop=True),stock_rank,
#                   df_prices_feat[df_prices_feat.Date.isin(val_dates)]['Target'].reset_index(drop=True)], axis=1)
#     sharpe=calc_spread_return_sharpe(df)
#     sharpe_ratio.append(sharpe)
#     print("Valid Sharpe: {}, RMSE: {}, MAE: {}".format(sharpe,rmse,mae))
    
#     del X_train, y_train,  X_val, y_val ##, pipeline_val_df, pipeline_X_df
#     gc.collect()
    
# print("\nAverage cross-validation Sharpe Ratio: {:.4f}, standard deviation = {:.2f}.".format(np.mean(sharpe_ratio),np.std(sharpe_ratio)))
# model_performance.append({"model":"RandomForestRegressor","Mean Sharpe Ratio":np.mean(sharpe_ratio),"SD Sharpe Ratio":std.mean(sharpe_ratio)})


In [ ]:

# feat_importance['avg'] = feat_importance.mean(axis=1)
# feat_importance = feat_importance.sort_values(by='avg',ascending=True)
# feat_importance = feat_importance.tail(20)
# pal=sns.color_palette("plasma_r", 29).as_hex()[2:]

# fig=go.Figure()
# for i in range(len(feat_importance.index)):
#     fig.add_shape(dict(type="line", y0=i, y1=i, x0=0, x1=feat_importance['avg'][i], 
#                        line_color=pal[::-1][i],opacity=0.7,line_width=4))
#     fig.add_trace(go.Scatter(x=feat_importance['avg'], y=feat_importance.index, mode='markers', 
#                              marker_color=pal[::-1], marker_size=8,
#                              hovertemplate='%{y} Importance = %{x:.0f}<extra></extra>'))
#     fig.update_layout(title='Overall Feature Importance', 
#                       xaxis=dict(title='Average Importance',zeroline=False),
#                       yaxis_showgrid=False, margin=dict(l=120,t=80),
#                       height=700, width=800)
# fig.show()


**Compare Performance from different models**

In [ ]:
for result in model_performance:
    print(result)

**Model selection argumentation**

XGBoost presented a better performance. However, the algorithm need considerably more time to be trained. Since this project was carried out using solely Kaggle due to the lack of a proper workstation, it was decided to go for the LGBMlight model. This way, the hyperparameter tuning time was reduced significantly.

### Hyperparameter optimization with Optuna

Once the best model is selected, the best hyperparameters are selected using Optuna. Optuna is a state-of-the-art Machine Learning hyperparameter optimization framework. 

More info about Optuna: https://optuna.org/

In [42]:
ts_fold = TimeSeriesSplit(n_splits=10,gap=10000)

In [43]:
sharpe_ratio=[]
def objective(trial):

    # Optuna suggest params
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 350, 1000),
        'num_leaves' : trial.suggest_int('n_estimators', 150, 400),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.10),
        'subsample': trial.suggest_uniform('subsample', 0.50, 0.90),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.50, 0.90),
       # 'metric': 'mae',
       # 'device': 'gpu',
       # 'gpu_platform_id': 0,
       # 'gpu_device_id': 0,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0)
    }
    """
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 350, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.10),
        'subsample': trial.suggest_uniform('subsample', 0.50, 0.90),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.50, 0.90)

    }
    """
    for fold, (train_idx, val_idx) in enumerate(ts_fold.split(X, y)):

        print(f"\n========================== Fold {fold+1} ==========================")

        X_train, y_train = X.iloc[train_idx,:], y[train_idx]
        X_valid, y_val = X.iloc[val_idx,:], y[val_idx]

        print("Train Date range: {} to {}".format(X_train.Date.min(),X_train.Date.max()))
        print("Valid Date range: {} to {}".format(X_valid.Date.min(),X_valid.Date.max()))


        ##### TRAIIIIIIIIIIN
        """
        pipeline_X_df = full_pipeline.fit_transform(X_train)
        pipeline_X_df = pd.DataFrame(pipeline_X_df, columns = num_attribs)

        pipeline_X_df.reset_index(drop = True, inplace=True)
        X_train.reset_index(drop = True, inplace=True)

        X_train.drop(num_attribs,axis=1,inplace=True)
        X_train = pd.concat([X_train,pipeline_X_df], axis=1)
      #  """
        X_train= X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

        ##### VALIDATION
        """
        pipeline_val_df = full_pipeline.transform(X_valid)
        pipeline_val_df = pd.DataFrame(pipeline_val_df, columns = num_attribs)
        pipeline_val_df.reset_index(drop = True, inplace=True)
        X_valid.reset_index(drop = True, inplace=True)
        X_valid.drop(num_attribs,axis=1,inplace=True)
        X_valid = pd.concat([X_valid,pipeline_val_df], axis=1)

       # """
        X_valid= X_valid.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

        X_train.drop(['Date','SecuritiesCode'], axis=1, inplace=True)
        X_val=X_valid[X_valid.columns[~X_valid.columns.isin(['Date','SecuritiesCode'])]]
        val_dates=X_valid.Date.unique()[1:-1]
        print("\nTrain Shape: {} {}, Valid Shape: {} {}".format(X_train.shape, y_train.shape, X_val.shape, y_val.shape))
        
        model = LGBMRegressor(**params)
        model.fit(X_train, y_train, 
                                      eval_set=[(X_train, y_train), (X_val, y_val)],
                                      verbose = False,
                                      eval_metric=['mae','mse'])
        y_pred = model.predict(X_val)
        """
        xgb = xgboost.XGBRegressor()
        xgb.fit(X_train, y_train)
        y_pred = xgb.predict(X_val)
        """
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        mae = mean_absolute_error(y_val, y_pred)
        #feat_importance["Importance_Fold"+str(fold)]=gbm.feature_importances_
        #feat_importance.set_index(X_train.columns, inplace=True)

        rank=[]
        X_val_df=X_valid[X_valid.Date.isin(val_dates)]
        for i in X_val_df.Date.unique():
            temp_df = X_val_df[X_val_df.Date == i].drop(['Date','SecuritiesCode'],axis=1)
            temp_df["pred"] = model.predict(temp_df)
            temp_df["Rank"] = (temp_df["pred"].rank(method="first", ascending=False)-1).astype(int)
            rank.append(temp_df["Rank"].values)

        stock_rank=pd.Series([x for y in rank for x in y], name="Rank")
        df=pd.concat([X_val_df.reset_index(drop=True),stock_rank,
                      df_prices_feat[df_prices_feat.Date.isin(val_dates)]['Target'].reset_index(drop=True)], axis=1)
        sharpe=calc_spread_return_sharpe(df)
        sharpe_ratio.append(sharpe)
        print("Valid Sharpe: {}, RMSE: {}, MAE: {}".format(sharpe,rmse,mae))

        del X_train, y_train,  X_val, y_val ##, pipeline_val_df, pipeline_X_df
        gc.collect()
        
    return np.mean(sharpe_ratio) #,np.std(sharpe_ratio)

**Select the best model, tune the parameters to find the best ones, retrain the WHOLE DATA with the best parameters**

In [ ]:
%%time
opt = optuna.create_study(direction='maximize',sampler=optuna.samplers.RandomSampler(seed=0))
opt.optimize(objective,n_trials=10)

# Save best parameters
trial = opt.best_trial
params_best = dict(trial.params.items())
params_best['random_seed'] = 0
    
# Create the model with best parameters
model_o = LGBMRegressor(**params_best)
#model_o = xgboost.XGBRegressor(**params_best)


[I 2022-06-22 18:58:59,798] A new study created in memory with name: no-name-0c48d0e4-0c7e-4b29-afa1-6bba5b813bbf



========================== Fold 1 ==========================
Train Date range: 2017-01-04 00:00:00 to 2017-06-12 00:00:00
Valid Date range: 2017-06-19 00:00:00 to 2017-12-01 00:00:00

Train Shape: (202033, 70) (202033,), Valid Shape: (212025, 70) (212025,)
Valid Sharpe: 0.029690761231897945, RMSE: 0.017238235208123513, MAE: 0.011213619574465043

========================== Fold 2 ==========================
Train Date range: 2017-01-04 00:00:00 to 2017-11-22 00:00:00
Valid Date range: 2017-12-01 00:00:00 to 2018-05-18 00:00:00

Train Shape: (414058, 70) (414058,), Valid Shape: (212025, 70) (212025,)
Valid Sharpe: 0.11358255610691652, RMSE: 0.020241641538484807, MAE: 0.01347203881647814

========================== Fold 3 ==========================
Train Date range: 2017-01-04 00:00:00 to 2018-05-11 00:00:00
Valid Date range: 2018-05-18 00:00:00 to 2018-10-26 00:00:00

Train Shape: (626083, 70) (626083,), Valid Shape: (212025, 70) (212025,)
Valid Sharpe: -0.1183789021283854, RMSE: 0.01990

In [ ]:
params_best

In [ ]:
#Shows scores for all trials
optuna.visualization.plot_optimization_history(opt)

In [ ]:
# Vizualize parameter importance
optuna.visualization.plot_param_importances(opt)

In [ ]:
X1 = X.drop(['Date','SecuritiesCode'], axis=1, inplace=False)
model_o.fit(X1,y)
#X.drop(['Date','SecuritiesCode'], axis=1, inplace=True)

In [ ]:
del df_prices_feat
gc.collect()

### 5. API Submission

In [ ]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

In [ ]:
%%time
cols=['Date','SecuritiesCode','Open','High','Low','Close','Volume','AdjustmentFactor']
stock_prices_df=stock_prices_df[stock_prices_df.Date>='2021-08-01'][cols]
cols_fin = col
cols_fin.remove("Target")
counter = 0
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    
    ## Loading API data and combine it with the dataset
    current_date = prices["Date"].iloc[0]
    if counter == 0:
        df_price_raw = stock_prices_df.loc[stock_prices_df["Date"] < current_date]
    
    df_price_raw = pd.concat([df_price_raw, prices[cols]]).reset_index(drop=True)
    
    ## Feature engineering
    df_price = adjust_price(df_price_raw)
    features = merge_metadata(df_price, stocks2000_df)  
    features = generate_feature(features)
    features = encoding(features) 
    features = features.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    features = features[X.columns.to_list()]
    
    ## --------- Pipeline
    """
    pipeline_feat_df = full_pipeline.transform(features)
    pipeline_feat_df = pd.DataFrame(pipeline_feat_df, columns = num_attribs)
    pipeline_feat_df.reset_index(drop = True, inplace=True)
    features.reset_index(drop = True, inplace=True)
    features.drop(num_attribs,axis=1,inplace=True)
    features = pd.concat([features,pipeline_feat_df], axis=1)
    """
    ## ---------
    
    feat = features[features.Date == current_date].drop(['SecuritiesCode','Date'],axis=1)
    display(feat)
    
    ## Prediction using the Model
    #feat["pred"] = gbm.predict(feat)
    
    ## Prediction using model with optimized paramat
    feat["pred"] = model_o.predict(feat)

    ## Generate Ranking 0-1999
    feat["Rank"] = (feat["pred"].rank(method="first", ascending=False)-1).astype(int)
    
    ## Input the ranking to the submission file
    sample_prediction["Rank"] = feat["Rank"].values
    display(sample_prediction.head())
    
    ## Input the ranking to the submission file
    assert sample_prediction["Rank"].notna().all()
    assert sample_prediction["Rank"].min() == 0
    assert sample_prediction["Rank"].max() == len(sample_prediction["Rank"]) - 1
    
    ## Submitt prediction file
    env.predict(sample_prediction)
    counter += 1